In [ ]:
import sys
from torch.utils.data import random_split
sys.path.append("../")

In [ ]:
from datasets import datasets
import pickle
import os
import pandas
from PIL import Image
import matplotlib.pyplot as plt
import numpy
import cv2

Loading training set information from the CSV file

In [ ]:
train_info = pandas.read_csv("../data/raw_data/train.csv")

Loading images from folders

In [ ]:
images = numpy.array()

for path in os.listdir("../data/raw_data/train_images"):
    img_file = Image.open(os.path.join("../data/raw_data/train_images", path))
    images.append(img_file)

train_info['image'] = images

Representing number of images for each individual class

In [ ]:
train_info.info()

In [ ]:
train_info['class'].value_counts()

In [ ]:
%%time

def to_rgb(img):
    if numpy.array(img).shape[2] == 4:
        img = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGRA2BGR)
    return Image.fromarray(numpy.array(img))

train_info['image'].apply(
    func=lambda image: to_rgb(image)
)

Noise Analysis

In [ ]:
def is_noisy(self, image, threshold: int = 10):
    scaled_img = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
    variance = cv2.Laplacian(src=scaled_img, threshold=threshold).var()
    return variance < threshold

In [ ]:
noisy_images = train_info[is_noisy(train_info['image'])]['image']

In [ ]:
plot, ax = plt.subplots()

for col in enumerate(noisy_images.shape[0]):
    for row in range(noisy_images.shape[0]):
        ax[col, row].imshow(noisy_images)
plt.show()

Removing image noise using smoothing filters

In [ ]:
def apply_median_filter(image: numpy.ndarray, kernel_size: int):
    """
    Function applies standard non-linear median filter 
    to image for removing salt-and-papper noise from image
    """
    if len(image) == 0: return 
    filtered_img = cv2.medianBlur(
        src=image,
        ksize=kernel_size
    )
    return filtered_img

def apply_bilateral_filtering(
    image: numpy.ndarray, 
    kernel_size: int,
    sigma_space: int, 
    sigma_color: int
):
    """
    Function applies bilateral filter to given image
    """
    smoothed_img = cv2.bilateralFilter(
        src=image,
        sigmaColor=sigma_color,
        sigmaSpace=sigma_space,
        d=kernel_size,
    )
    return smoothed_img

In [ ]:
for idx, image in enumerate(noisy_images.to_numpy()):
    # Applying filter to noisy image
    blurred_img = apply_bilateral_filtering(
        image=image,
        sigma_color=30,
        sigma_space=30,
        kernel_size=5
    )
    train_info.iloc[idx, 'image'] =  blurred_img


Forming Data Augmentation Instructions

In [ ]:
HEIGHT = 512
WIDTH = 512

In [ ]:
from torchvision import transforms 
from PIL import Image


train_transformations = [
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((HEIGHT, WIDTH), interpolation=Image.NEAREST),
    transforms.RandomAdjustSharpness(sharpness_factor=1.3, p=0.5),
]

eval_transformations = [
    transforms.ToTensor(),
    transforms.Resize((HEIGHT, WIDTH), interpolation=Image.NEAREST),
]

Splitting dataset

In [ ]:
from torch.utils.data import random_split

train_size = int(train_info.shape[0] * 0.7) # 70% of the data will be in training set
evaluation_size = train_info.shape[0] - train_size # rest of 30% will be in evaluation set

train_d, test_d = random_split(
    train_info, 
    [train_size, evaluation_size]
)

Splitting data

In [ ]:
# training set
train_dataset = datasets.FaceRecognitionDataset(
    images=numpy.array(train_info['image'])[train_d.indices],
    labels=numpy.array(train_info['class'])[train_d.indices]
)

# evaludation dataset
test_dataset = datasets.FaceRecognitionDataset(
    images=numpy.array(train_info['image'])[test_d.indices],
    labels=numpy.array(train_info['class'])[test_d.indices]
)

Image Visualization

In [ ]:
def visualize_k_random_images(images, k):
    """
    Function visualizes k images, extracted from given source
    
    Args:
        images - typing.List[str] - array of images
        k - number of images
    """
    _, ax = plt.subplots(2, images // k)
    k_random_images = numpy.random.choice(size=2 * images // k, a=images)
    for col in range(ax.shape[0]):
        for row in range(ax.shape[0]):
            ax[col, row].imshow(k_random_images.pop())
        
visualize_k_random_images(train_dataset.images)

Texture Analysis

In [ ]:
def local_binary_patterns(img):
    pass 

def local_ternary_patterns(img):
    pass 

def local_phase_quantization(img):
    pass

In [ ]:
def gabor_filtering():
    pass

Evaluating Transformed and Recovered Images

In [ ]:
from skimage.metrics import ssim, ncc

def ssim_score(orig_img, blur_img):
    return ssim(orig_img, blur_img)

def normalized_cross_correlation(orig_img, blur_img):
    return ncc(orig_img, blur_img)

In [ ]:
imgs = train_info['image'].to_numpy()

for idx, image in zip(train_dataset.dataset_idxs, train_dataset.images):
    ssim = ssim_score(imgs[idx], image)
    cc = normalized_cross_correlation(imgs[idx], image)
    print('ssim - %s; normalized cross correlation - %s;' % ssim, cc)


Saving datasets to pickle format

In [ ]:
pickle.dumps(train_dataset, "../data/processed_data/training_set.pkl")
pickle.dumps(test_dataset, "../data/processed_data/testing_set.pkl")